In [1]:
import bw2data as bd
import bw2io as bi

In [2]:
PROJECT_NAME = "Linking dMFA & LCA"

bi.restore_project_directory(
    fp='/etc/data/ecoinvent-3.10-cutoff-bw25.tar.gz',
    project_name=PROJECT_NAME,
    overwrite_existing=True
)
bd.projects.set_current(PROJECT_NAME)

db=bd.Database('ecoinvent-3.10-cutoff')

# selecting ecoinvent processes
primary_steel_production = bd.get_node(
    name='steel production, electric, chromium steel 18/8',
    location = 'RER').copy(name = 'primary steel production', database=db.name)

secondary_steel_production = bd.get_node(
    name='steel production, electric, chromium steel 18/8',
    location = 'RER').copy(name = 'secondary steel production', database=db.name)

ecoinvent_steel_scrap = bd.get_node(name='market for scrap steel', location = 'Europe without Switzerland')

ecoinvent_iron_ore = bd.get_node(name='market for iron ore concentrate', location = 'GLO')

# electric arc furnace is loaded with 60% scrap and 40% virgin raw materials

Restoring project backup archive - this could take a few minutes...
Restored project: Linking dMFA & LCA
11:48:59+0000 [info     ] Applying automatic update: 4.0 database search directories FTS5
11:48:59+0000 [info     ] Reindexing database ecoinvent-3.10-biosphere
11:48:59+0000 [info     ] Reindexing database ecoinvent-3.10-cutoff


In [3]:
### creating new output product for recycling process

primary_steel = db.new_node(
    name='primary steel',
    code = 'primary steel',
    unit="kilogram",
    location = "RER",
    type=bd.labels.product_node_default,
)
primary_steel.save()

secondary_steel = db.new_node(
    name='secondary steel',
    code = 'secondary steel',
    unit="kilogram",
    location = "RER",
    type=bd.labels.product_node_default,
)
secondary_steel.save()

In [4]:
#deleting old output in primary production
for exc in primary_steel_production.production():
    exc.delete()

#adding the secondary steel output
primary_steel_production.new_edge(
    input=primary_steel,
    amount=1,
    type=bd.labels.production_edge_default
).save()

#checking that the output was replaced
print(list(primary_steel_production.production()))

#deleting old output in secondary production
for exc in secondary_steel_production.production():
    exc.delete()

#adding the secondary steel output
secondary_steel_production.new_edge(
    input=secondary_steel,
    amount=1,
    type=bd.labels.production_edge_default
).save()

#checking that the output was replaced
list(secondary_steel_production.production())

[Exchange: 1 kilogram 'primary steel' (kilogram, RER, None) to 'primary steel production' (kilogram, RER, None)>]


[Exchange: 1 kilogram 'secondary steel' (kilogram, RER, None) to 'secondary steel production' (kilogram, RER, None)>]

In [5]:
#changing material inputs in primary steel production (replace iron scrap with iron ore)
iron_scrap_input_primary = list(primary_steel_production.edges())[13]
iron_ore_input_amount = iron_scrap_input_primary['amount']
iron_scrap_input_primary.delete() #new inflow is added in cell below
iron_scrap_input_primary.save()

#changing material inputs in secondary steel production (iron, nickel & chromium inflows at 40%)
chromium_input = list(secondary_steel_production.edges())[9]
nickel_input = list(secondary_steel_production.edges())[10]
iron_scrap_input_secondary = list(secondary_steel_production.edges())[13]

steel_scrap_amount = (nickel_input['amount'] + chromium_input['amount'] + iron_scrap_input_secondary['amount']) * 0.6

nickel_input['amount'] = nickel_input['amount'] * 0.4
chromium_input['amount'] = chromium_input['amount'] * 0.4
iron_scrap_input_secondary['amount'] = iron_scrap_input_secondary['amount'] * 0.4

chromium_input.save()
nickel_input.save()
iron_scrap_input_secondary.save()

In [6]:
#adding iron ore as inflow for primary steel production
primary_steel_production.new_edge(
    amount = steel_scrap_amount, #calculated in cell above
    type=bd.labels.consumption_edge_default,
    input=ecoinvent_iron_ore
).save()

#adding 60% of raw material inputs as scrap metal for secondary steel production
secondary_steel_production.new_edge(
    amount = steel_scrap_amount, #calculated in cell above
    type=bd.labels.consumption_edge_default,
    input=ecoinvent_steel_scrap
).save()

In [8]:
list(primary_steel_production.technosphere())

[Exchange: 0.004 kilogram 'market for anode, for metal electrolysis' (kilogram, GLO, None) to 'primary steel production' (kilogram, RER, None)>,
 Exchange: 0.032 kilogram 'market for argon, liquid' (kilogram, RER, None) to 'primary steel production' (kilogram, RER, None)>,
 Exchange: 4e-11 unit 'market for electric arc furnace converter' (unit, GLO, None) to 'primary steel production' (kilogram, RER, None)>,
 Exchange: -2.8657850546607665e-06 kilogram 'market for electric arc furnace dust' (kilogram, CH, None) to 'primary steel production' (kilogram, RER, None)>,
 Exchange: -0.019124534214945338 kilogram 'market for electric arc furnace dust' (kilogram, Europe without Switzerland, None) to 'primary steel production' (kilogram, RER, None)>,
 Exchange: -0.01924 kilogram 'market for electric arc furnace secondary metallurgy slag' (kilogram, RER, None) to 'primary steel production' (kilogram, RER, None)>,
 Exchange: -0.11089719292129314 kilogram 'market for electric arc furnace slag' (kilo